# 소개 (Introduction)

**미국 청소년 종단 연구 (National Longitudinal Survey of Youth) 1997-2011 데이터셋**은 미국 데이터를 다루는 사회 과학자들에게 가장 중요한 데이터베이스 중 하나입니다.

이 데이터셋을 통해 과학자들은 **소득의 결정 요인**뿐만 아니라 **교육적 성취**에 대해서도 연구할 수 있으며, 이는 **정부 정책**과 밀접한 관련이 있습니다. 또한, **다양한 인종, 성별 및 기타 요인에 따른 교육적 성취와 급여 수준의 차이**와 같은 **정치적으로 민감한 이슈**를 조명할 수도 있습니다.

이 변수들이 **교육과 소득에 미치는 영향을 더 잘 이해하게 되면**, 보다 적절한 **정부 정책**을 수립할 수도 있을 것입니다.

<center><img src=https://i.imgur.com/cxBpQ3I.png height=400></center>


# Plotly 업그레이드

In [7]:
%pip install --upgrade plotly

Note: you may need to restart the kernel to use updated packages.


###  라이브러리 불러오기 (Import Statements)



In [8]:
import pandas as pd
import numpy as np

import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

## 노트북 출력 설정 (Notebook Presentation)

In [9]:
pd.options.display.float_format = '{:,.2f}'.format

# 데이터 불러오기 (Load the Data)



In [10]:
df_data = pd.read_csv('NLSY97_subset.csv')

### 데이터 이해하기 (Understand the Dataset)

다음 파일을 확인하세요: **NLSY97_Variable_Names_and_Descriptions.csv**

---------------------------

    :주요 변수(Key Variables):
      1. S           - 학력연수 (Years of schooling (highest grade completed as of 2011))
      2. EXP         - 총 근무연수 (Total out-of-school work experience (years) as of the 2011 interview.)
      3. EARNINGS    - 시간당 소득 (Current hourly earnings in $ reported at the 2011 interview)

# 기초 데이터 탐색 🔍 (Preliminary Data Exploration)

**문제 확인 (Challenge)**

* df_data의 형태는 어떻게 되는가?(What is the shape of `df_data`? )
* 몇 개의 행과 열이 있는가? (How many rows and columns does it have?)
* 컬럼(변수) 이름은 무엇인가? (What are the column names?)
* NaN 값 또는 중복 데이터가 있는가? (Are there any NaN values or duplicates?)

## 데이터 정리 - 결측값 및 중복 데이터 확인 (Data Cleaning - Check for Missing Values and Duplicates)

중복된 행을 찾아 제거하세요. (Find and remove any duplicate rows.)

## 기술 통계 (Descriptive Statistics)

## 특징 시각화 (Visualise the Features)

# 훈련 & 테스트 데이터셋 분할 (Split Training & Test Dataset)

우리는 전체 데이터를 학습에 사용할 수 없습니다.
데이터의 20%를 테스트 데이터셋으로 분리하여 나중에 모델 성능을 평가하는 데 사용합니다.
<br>
We *can't* use all the entries in our dataset to train our model. Keep 20% of the data for later as a testing dataset (out-of-sample data).  

# 단순 선형 회귀 (Simple Linear Regression)

학교 교육 연수(S)만을 이용하여 소득(EARNINGS)을 예측하는 회귀 모델을 학습합니다.
sklearn을 활용하여 훈련 데이터에 대해 회귀 분석을 수행하고, 결정 계수(R²)를 확인하세요.
<br>
Only use the years of schooling to predict earnings. Use sklearn to run the regression on the training dataset. How high is the r-squared for the regression on the training data? 

### 모델 계수 평가 (Evaluate the Coefficients of the Model)

회귀 계수(기울기)가 예상한 부호(양수 또는 음수)를 갖는지 확인하세요.
<br>
추가적인 학업 연수가 소득에 미치는 영향을 해석해 봅시다.
<br>
Here we do a sense check on our regression coefficients. The first thing to look for is if the coefficients have the expected sign (positive or negative).
<br>
Interpret the regression. How many extra dollars can one expect to earn for an additional year of schooling?

### 예측값 및 회귀 잔차 분석 (Analyse the Estimated Values & Regression Residuals)

회귀 모델의 성능은 **잔차(Residuals)**, 즉 **모델의 예측값(ŷᵢ)과 실제값(yᵢ) 간의 차이**에 따라 평가됩니다.
<br>
잔차의 분포에서 어떤 패턴이 보이나요?
<br>
How good our regression is also depends on the residuals - the difference between the model's predictions ( 𝑦̂ 𝑖 ) and the true values ( 𝑦𝑖 ) inside y_train. Do you see any patterns in the distribution of the residuals?

# 다변량 회귀 (Multivariable Regression)

이제 학력 연수(S)와 총 근무 경력(EXP)을 동시에 사용하여 소득(EARNINGS)을 예측하는 회귀 모델을 학습합니다.
<br>
훈련 데이터에서 **결정 계수(R²)**가 얼마나 높은지 확인하세요.
<br>
Now use both years of schooling and the years work experience to predict earnings. How high is the r-squared for the regression on the training data? 

### 모델 계수 평가 (Evaluate the Coefficients of the Model)

### 예측값 및 회귀 잔차 분석 (Analyse the Estimated Values & Regression Residuals)

# 모델을 사용하여 예측하기 (Use Your Model to Make a Prediction)

**2011년 기준, 학사 학위(12 + 4년의 교육)와 5년의 근무 경험이 있는 사람은 얼마를 벌 것으로 예상될까요?**
<br>
How much can someone with a bachelors degree (12 + 4) years of schooling and 5 years work experience expect to earn in 2011?

# 추가 실험 및 심화 탐구 (Experiment and Investigate Further)

회귀 분석을 개선하여 소득 예측을 더 정확하게 만들기 위해 어떤 추가적인 특징(Feature)을 고려할 수 있을까요?
<br>
Which other features could you consider adding to further improve the regression to better predict earnings? 